# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [9]:
spaceship.isnull().sum()

spaceship.dropna(inplace=True)

In [10]:
spaceship['Cabin'] = spaceship['Cabin'].str[0]

In [11]:
spaceship.drop(columns=["PassengerId", "Name"], inplace=True)

encoded_df = pd.get_dummies(spaceship, columns=['HomePlanet',"CryoSleep","Destination", "VIP", 'Cabin'], dtype=int)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [12]:
features = encoded_df.drop(columns = ['Transported'])
target = encoded_df['Transported']

**Perform Train Test Split**

In [14]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [26]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

pred = knn.predict(X_test)
pred

y_test.values

array([ True, False,  True, ...,  True,  True,  True])

In [16]:
knn.score(X_test, y_test)

0.75642965204236

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold 
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score

In [32]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [33]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [34]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,VIP_False,VIP_True,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
0,0.405063,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.050633,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.379747,0.00000,0.007916,0.000000,0.051276,0.000000,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.215190,0.00131,0.000000,0.046111,0.016378,0.000049,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.329114,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
X_test_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,...,VIP_False,VIP_True,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
0,0.632911,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.227848,0.0,0.000000,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.189873,0.0,0.000000,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.658228,0.0,0.000000,0.0,0.00000,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.784810,0.0,0.054775,0.0,0.07774,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


- Bagging and Pasting

In [36]:
# Initialize the Bagging classifier with Decision Tree as base estimator
bagging_class = BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=20),
                                  n_estimators=100,
                                  max_samples=1000)

# Train the Bagging classifier
bagging_class.fit(X_train_norm, y_train)

# Make predictions
pred = bagging_class.predict(X_test_norm)

# Calculate and print evaluation metrics
print("Accuracy:", accuracy_score(y_test, pred))
print("Recall:", recall_score(y_test, pred, average='weighted'))  # Use 'weighted' for multiclass
print("Precision:", precision_score(y_test, pred, average='weighted', zero_division=0))  # Use 'weighted' for multiclass

Accuracy: 0.7776096822995462
Recall: 0.7776096822995462
Precision: 0.7776325577669181


- Random Forests

In [37]:
forest = RandomForestClassifier(n_estimators=100, max_samples=1000)

# Train the Random Forest classifier
forest.fit(X_train_norm, y_train)

# Make predictions
pred = forest.predict(X_test_norm)

# Calculate and print evaluation metrics
print("Accuracy:", accuracy_score(y_test, pred))
print("Recall:", recall_score(y_test, pred, average='weighted'))  # Use 'weighted' for multiclass
print("Precision:", precision_score(y_test, pred, average='weighted', zero_division=0))  # Use 'weighted' for multiclass

Accuracy: 0.7851739788199698
Recall: 0.7851739788199698
Precision: 0.7855504245493818


- Gradient Boosting

In [39]:
gb_reg = GradientBoostingClassifier(max_depth=20, n_estimators=100)

# Train the Gradient Boosting classifier
gb_reg.fit(X_train_norm, y_train)

# Make predictions
pred = gb_reg.predict(X_test_norm)

# Calculate and print evaluation metrics
print("Accuracy:", accuracy_score(y_test, pred))
print("Recall:", recall_score(y_test, pred, average='weighted'))  # Use 'weighted' for multiclass
print("Precision:", precision_score(y_test, pred, average='weighted', zero_division=0))  # Use 'weighted' for multiclass

Accuracy: 0.794251134644478
Recall: 0.794251134644478
Precision: 0.7948585045215711


- Adaptive Boosting

In [38]:
ada_reg = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20),
                            n_estimators=100)

# Train the Random Forest classifier
ada_reg.fit(X_train_norm, y_train)

# Make predictions
pred = ada_reg.predict(X_test_norm)

# Calculate and print evaluation metrics
print("Accuracy:", accuracy_score(y_test, pred))
print("Recall:", recall_score(y_test, pred, average='weighted'))  # Use 'weighted' for multiclass
print("Precision:", precision_score(y_test, pred, average='weighted', zero_division=0))  # Use 'weighted' for multiclass

Accuracy: 0.7760968229954615
Recall: 0.7760968229954615
Precision: 0.7768292542999093


Which model is the best and why?

The gradient was the model that produced the best results.